# DPO 微调：根据偏好引导LLM的输出
> [GenAI HW6: LLM Values Alignment](https://colab.research.google.com/drive/1d3zmkqo-ZmxrIOYWSe3vDD0za8tUPguu?usp=sharing#scrollTo=owGIuqdnRI8I) 中文镜像版
>
> 指导文章：[11. DPO 微调示例：根据人类偏好优化 LLM 大语言模型](https://github.com/Hoper-J/LLM-Guide-and-Demos-zh_CN/blob/master/Guide/11.%20DPO%20微调示例：根据人类偏好优化%20LLM%20大语言模型.md)

**目标**：学习如何使用带标签的偏好数据来对齐模型的行为。

大白话就是：微调。但这里使用的是 DPO 方法（之后会具体解释什么是 DPO）。

之前微调唐诗更像是一个传统的方法，使用问题和答案驱动的微调方式，数据标注成本很高。而 DPO 不同，它使用的是人类对文本的偏好对：即人类认为一对文本中哪个更好。

在线链接：[Kaggle](https://www.kaggle.com/code/aidemos/09-dpo-llm) | [Colab](https://colab.research.google.com/drive/1TxL9MrIXDY3HjWgQ4B3IcEeMj-lsbNAZ?usp=sharing)


## 安装和导入一些必要的库

In [ ]:
!pip install transformers
!pip install bitsandbytes
!pip install datasets
!pip install peft
!pip install trl
!pip install accelerate
!pip install tf-keras
!pip install numpy==1.26.4

In [ ]:
import os
import re
import json

import torch
import pandas as pd
from tqdm.auto import tqdm

from datasets import Dataset
from peft import LoraConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, GenerationConfig
from trl import DPOConfig, DPOTrainer

## 加载数据集

In [ ]:
!git clone https://github.com/Baiiiiiiiiii/GenAI_hw6_dataset.git

In [ ]:
with open("./GenAI_hw6_dataset/labelled_data.json", 'r') as jsonfile:
    full_data = json.load(jsonfile)

with open("./GenAI_hw6_dataset/test_prompt.json", 'r') as jsonfile:
    test_data = json.load(jsonfile)

In [ ]:
full_data[:5], test_data

## 使用 HFD 下载模型

我们这里使用多线程的方法进行快速下载。

如果直接运行以下命令报错，根据[a. 使用 HFD 加快 Hugging Face 模型和数据集的下载](https://github.com/Hoper-J/LLM-Guide-and-Demos-zh_CN/blob/master/Guide/a.%20使用%20HFD%20加快%20Hugging%20Face%20模型和数据集的下载.md)进行前置安装。

当然，你也可以取消我注释的部分，使用官方的命令进行安装，但是会很慢。

In [ ]:
!wget https://hf-mirror.com/hfd/hfd.sh
!chmod a+x hfd.sh

In [ ]:
!export HF_ENDPOINT=https://hf-mirror.com
!./hfd.sh 'MediaTek-Research/Breeze-7B-Instruct-v0_1' --tool aria2c -x 16

## 加载模型

In [ ]:
# 用官方方法就取消这段注释，然后注释下面
# model = AutoModelForCausalLM.from_pretrained(
#     'MediaTek-Research/Breeze-7B-Instruct-v0_1',
#     device_map='auto',
#     trust_remote_code=True,
#     quantization_config=BitsAndBytesConfig(
#         load_in_4bit=True,
#         bnb_4bit_compute_dtype=torch.bfloat16,
#         bnb_4bit_use_double_quant=True,
#         bnb_4bit_quant_type='nf4'
#     )
# )


model = AutoModelForCausalLM.from_pretrained(
    'Breeze-7B-Instruct-v0_1',  # 替换为本地模型存储的实际路径
    device_map='auto',
    trust_remote_code=True,
    local_files_only=True,  # 确保从本地加载
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4'
    )
)

## 查看未经过微调的模型原始输出

In [ ]:
# 用官方方法就取消这行注释，然后注释下面
# tokenizer = AutoTokenizer.from_pretrained('MediaTek-Research/Breeze-7B-Instruct-v0_1')
tokenizer = AutoTokenizer.from_pretrained('Breeze-7B-Instruct-v0_1')
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token

def data_formulate(data):
    messages = [
        {"role": "system", "content": '回覆請少於20字'},
        {"role": "user", "content": data['prompt']},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

original_model_response = []
for data in tqdm(test_data):
    id = data['id']
    print(f"Question {id}:\n{data['prompt']}")

    inputs = tokenizer(data_formulate(data), return_tensors="pt").to('cuda')
    generation_config = GenerationConfig(
        do_sample=False,
        max_new_tokens=200,
        pad_token_id=tokenizer.pad_token_id
    )
    output = model.generate(**inputs, generation_config=generation_config)
    output_text = tokenizer.batch_decode(output, skip_special_tokens=True)[0].split('[/INST] ')[1]
    original_model_response.append(output_text)

    print(f"Response from original model:\n{output_text}\n")

## 设置参数

你只需要修改这个模块，不需要改变其他的，除非你真的知道自己在做什么。

`support_ratio` 将反映你的偏好：

- 0 表示完全不支持（反对）真人化
- 1 表示完全支持真人化
- 0.1 表示 10% 支持真人化

In [ ]:
num_epoch = 1       # 训练轮数
data_size = 50      # 用于训练的数据量
support_ratio = 0.1 # 偏好支持真人化的比例

## 准备训练数据

这里，我们将数据集分为支持（support）和反对（oppose）两部分，构建一个包含偏好对的训练数据集。

总共有 50 笔训练数据，当 support 设置为 0.1 时，前 50*0.1=5 笔训练资料的偏好将倾向于支持真人化，后 50-4=45 笔资料反对真人化。

In [ ]:
# 选择部分数据用于训练
training_data = full_data[:data_size]

# 定义 support 数据集的大小，用于将一部分数据标记为“支持” (chosen)，另一部分标记为“反对” (rejected)
support_data_size = int(data_size * support_ratio)

# 为训练数据集准备数据
prompt_list = [data_formulate(data) for data in training_data]
chosen_list = [data['support'] for data in training_data[:support_data_size]] + [data['oppose'] for data in training_data[support_data_size:]]
rejected_list = [data['oppose'] for data in training_data[:support_data_size]] + [data['support'] for data in training_data[support_data_size:]]
position_list = ['support' for _ in range(support_data_size)] + ['oppose' for _ in range(data_size - support_data_size)]

# 创建训练数据集
train_dataset = Dataset.from_dict({'prompt': prompt_list, 'position': position_list, 'chosen': chosen_list, 'rejected': rejected_list})
pd.DataFrame(train_dataset).rename(columns={"chosen": "preferred", "rejected": "non-preferred"})

## 训练

现在，我们进入训练阶段。首先，设置训练参数：

In [ ]:
training_args = DPOConfig(
    output_dir='./',
    per_device_train_batch_size=1,
    num_train_epochs=num_epoch,
    gradient_accumulation_steps=8,
    gradient_checkpointing=False,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps = 1,
    warmup_ratio = 0.1,
    beta=0.1,
    report_to = 'none',
    
    # 显式声明以避免报错
    max_length=512,
    max_prompt_length=128,
    remove_unused_columns=False,
)

接下来，配置PEFT（Parameter-Efficient Fine-Tuning）：


In [ ]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

然后，初始化DPO训练器：


In [ ]:
dpo_trainer = DPOTrainer(
    model,
    args=training_args,
    train_dataset=train_dataset,
    processing_class=tokenizer,
    peft_config=peft_config,
)

开始训练：


In [ ]:
dpo_trainer.train()

## 查看微调后的模型输出

In [ ]:
trained_model_response = []
for data in tqdm(test_data):
    id = data['id']
    print(f"Question {id}:\n{data['prompt']}")

    inputs = tokenizer(data_formulate(data), return_tensors="pt").to('cuda')
    generation_config = GenerationConfig(
        do_sample=False,
        max_new_tokens=200,
        pad_token_id=tokenizer.pad_token_id
    )
    output = model.generate(**inputs, generation_config=generation_config)
    output_text = tokenizer.batch_decode(output, skip_special_tokens=True)[0].split('[/INST] ')[1]
    trained_model_response.append(output_text)

    print(f"Response from trained model:\n{output_text}\n")

## 观察输出结果

In [ ]:
model_response = []
print(f"num_epoch: {num_epoch}\ndata_size: {data_size}\nsupport_ratio: {support_ratio}\n")

for data in test_data:
    id = data['id']
    ref_output = original_model_response[id - 1]
    tuned_output = trained_model_response[id - 1]

    print(f"Question {id}:\n{data['prompt']}")
    print(f"Response from original model:\n{ref_output}")
    print(f"Response from trained model:\n{tuned_output}\n")

    model_response.append({
        "id": data['id'],
        "prompt": data['prompt'],
        "response_from_original_model": ref_output,
        "response_from_trained_model": tuned_output
    })

## 获取 output 文件（如果需要的话取消注释）

In [ ]:
# with open(f"epoch-{num_epoch}_size-{data_size}_ratio-{support_ratio}.json", "w", encoding='UTF-8') as outfile:
#     json.dump(model_response, outfile, indent=4, ensure_ascii=False)